# Eléments de programmation

Pour mener à bien un calcul algorithmique le nombre d'éléments de langage n'est pas très important et peut se résumer aux 3 syntaxes suivantes

* Le choix conditionnel **if**-**elseif**-**else**-**end**.
* La boucle **for**-**end**.
* La boucle **while**-**end**

# if ... elseif ... else ... end

Un choix simple si le test est vrai (k==1) alors le bloc d'instruction est évalué
<!-- #endregion -->

In [2]:
k = 1
if k == 1
    println("k=1")
end

k=1


Le **else** permet de donner un résultat par défaut...

In [4]:
k = 1
if k != 1
    println("k<>1")
else
    println("k=1")
end

k=1


Une succession de **elseif** permet de choisir parmi plusieurs critères, dans la succession des blocs de **if** et **elseif** le premier qui est "vrai" est évaluer et l'instruction s'arrète.

In [6]:
k=2
if k==1
    println("k=1")
elseif k>1
    println("k>1")
else 
    println("k<1")
end

k>1


# La boucle for

Elle peut se définir à l'aide d'itérateurs ou de tableaux de valeurs les syntaxes "=" , "in" ou "$\in$" sont équivalentes

In [7]:
for i = 1:10
    println(i)
end

1
2
3
4
5
6
7
8
9
10


In [11]:
for i in ["un" 2 im]
    println(typeof(i))
end

String
Int64
Complex{Bool}


In [18]:
typeof.(["un", 2, im])

3-element Vector{DataType}:
 String
 Int64
 Complex{Bool}

In [19]:
eltype(["un" 2 im])

Any

La commande **break** permet de sortir d'une boucle à tout moment

In [20]:
for i = 1:1000
    println(i)
    if i >= 5
       break
    end
end

1
2
3
4
5


La commande **continue** permet elle de courtcircuiter la boucle en court et de passer à la valeur suivante

In [21]:
for i = 1:10
    if i % 3 != 0 # i modulo 3 different de 0
        continue
    end
    println(i)
end

3
6
9


# La boucle while

Tant que le test est "vrai" le bloc est évalué, le test se faisant en entrée de bloc

In [22]:
k = 0
while k<10
    k += 1  # k=k+1
    println(k)
end

1
2
3
4
5
6
7
8
9
10


De même que la boucle **for** les commandes **break** et **continue** sont valables ...

In [23]:
k=0
while k < 1000
    k += 1  # k=k+1
    if k % 5 != 0 # k modulo 5 diffèrent de 0
        continue # retour en début de boucle avec de nouveau un test sur k
    end
    println(k)
    if k > 30
        break
    end
end

5
10
15
20
25
30
35


In [34]:
let 
    n = 10000000
    k = 0
    while n > 1
        if n % 2 == 0
            n = n ÷ 2
        else
            n = 3n+1
        end
        k += 1
    end
    println(n)
    println("k \t = \t $k")
end

1
k 	 = 	 145


# Un peu d'optimisation

*Julia* est dit un langage performant, regardons rapidement quelques exemples à faire ou ne pas faire.

## Exemple de préallocation et utilisation de push!

In [36]:
n = 100000
@time begin
    A = zeros(0) # Tableau vide qui contiendra des Float64
    for i = 1:n
        A = [A; i]   # a chaque itération on change la taille de A
    end
end

 24.233625 seconds (496.93 k allocations: 37.266 GiB, 17.33% gc time)


In [37]:
typeof(A)

Vector{Float64} (alias for Array{Float64, 1})

In [38]:
A

100000-element Vector{Float64}:
      1.0
      2.0
      3.0
      4.0
      5.0
      6.0
      7.0
      8.0
      9.0
     10.0
     11.0
     12.0
     13.0
      ⋮
  99989.0
  99990.0
  99991.0
  99992.0
  99993.0
  99994.0
  99995.0
  99996.0
  99997.0
  99998.0
  99999.0
 100000.0

In [41]:
@time begin
    A = zeros(0)
    for i = 1:n
        push!(A,i)   # a chaque itération on change la taille de A
    end
end

  0.017135 seconds (298.99 k allocations: 7.916 MiB)


In [42]:
A

100000-element Vector{Float64}:
      1.0
      2.0
      3.0
      4.0
      5.0
      6.0
      7.0
      8.0
      9.0
     10.0
     11.0
     12.0
     13.0
      ⋮
  99989.0
  99990.0
  99991.0
  99992.0
  99993.0
  99994.0
  99995.0
  99996.0
  99997.0
  99998.0
  99999.0
 100000.0

In [44]:
@time begin
    A = zeros(Int64,n)
    for i in eachindex(A)
        A[i] = i
    end
end

  0.010592 seconds (298.98 k allocations: 6.851 MiB)


In [47]:
@time begin
    A = zeros(Int64,n)
    fill!(A, 1)
end

  0.000141 seconds (2 allocations: 781.297 KiB)


100000-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [49]:
@time begin
    A = [i for i=1:n]
end

  0.000121 seconds (4 allocations: 781.359 KiB)


100000-element Vector{Int64}:
      1
      2
      3
      4
      5
      6
      7
      8
      9
     10
     11
     12
     13
      ⋮
  99989
  99990
  99991
  99992
  99993
  99994
  99995
  99996
  99997
  99998
  99999
 100000

In [50]:
typeof(A)

Vector{Int64} (alias for Array{Int64, 1})

Cet exemple montre le coût prohibitif d'une réallocation dynamique qui impose une recopie totale de A à chaque itération.

## Exemple de vectorisation

Regardons la vectorisation sous *Julia* à l'aide de la construction d'une matrice de Vandermonde


In [64]:
@time begin
    n = 3000
    x = LinRange(0, 1, n)
    V = zeros(n,n)
    for i = 1:n
        V[:,i] .= x.^(i-1) # calcul vectorisé
    end
end

  0.339490 seconds (35.94 k allocations: 69.671 MiB, 8.98% gc time)


In [71]:
@time begin
    n = 3000
    x = LinRange(0, 1, n)
    V = zeros(n,n)
    for j=1:n
        for i=1:n
            V[i,j] = x[i]^(j-1) # calcul dévectorisé
        end
    end
end

  1.886602 seconds (49.41 M allocations: 960.007 MiB, 7.45% gc time)


In [61]:
typeof(X)

Matrix{Float64} (alias for Array{Float64, 2})

In [75]:
@time begin 
    n = 3000
    x = LinRange(0, 1, n)
    W = [x[i]^(j-1) for j=1:n, i=1:n]
end

  0.989547 seconds (33.01 M allocations: 575.400 MiB, 7.31% gc time, 6.61% compilation time)


3000×3000 Matrix{Float64}:
 1.0  1.0          1.0          …  1.0        1.0       1.0       1.0
 0.0  0.000333444  0.000666889     0.999      0.999333  0.999667  1.0
 0.0  1.11185e-7   4.44741e-7      0.998      0.998667  0.999333  1.0
 0.0  3.70741e-11  2.96593e-10     0.997002   0.998001  0.999     1.0
 0.0  1.23622e-14  1.97794e-13     0.996005   0.997335  0.998667  1.0
 0.0  4.12209e-18  1.31907e-16  …  0.995008   0.99667   0.998334  1.0
 0.0  1.37449e-21  8.79673e-20     0.994013   0.996005  0.998001  1.0
 0.0  4.58316e-25  5.86644e-23     0.993019   0.995341  0.997668  1.0
 0.0  1.52823e-28  3.91226e-26     0.992025   0.994677  0.997336  1.0
 0.0  5.09579e-32  2.60905e-29     0.991033   0.994014  0.997003  1.0
 0.0  1.69916e-35  1.73994e-32  …  0.990042   0.993351  0.996671  1.0
 0.0  5.66577e-39  1.16035e-35     0.989051   0.992689  0.996338  1.0
 0.0  1.88922e-42  7.73824e-39     0.988062   0.992027  0.996006  1.0
 ⋮                              ⋱                              

In [57]:
typeof(W)

Matrix{Float64} (alias for Array{Float64, 2})

In [58]:
function Vander(n)
    x = LinRange(0, 1, n)
    V = zeros(n, n)
    for i=1:n
        #V[:,i]=x.^(i-1)
        for j=1:n
            V[i,j]=x[i]^(j-1) # calcul dévectorisé
        end
    end
    return V
end
@elapsed Z = Vander(3000)

0.184000873

In [59]:
@elapsed Z = Vander(3000)

0.206228317

In [60]:
@elapsed Z = Vander(3000) 

0.179519609

In [64]:
function Vander2(n)
    x = LinRange(0, 1, n)
    return [x[i]^(j-1) for i=1:n, j=1:n]
end
@elapsed Z2=Vander2(3000)

0.314779955

In [66]:
@elapsed Z2=Vander2(3000)

0.293058917

In [67]:
typeof(Z2)

Matrix{Float64} (alias for Array{Float64, 2})

La conclusion n'est pas toujours évidente a priori... Néanmoins on
peut voir que le fait de mettre le code dans une fonction impose
une optimisation à la compilation et une contextualisation du type
et que l'écriture explicite des boucles donne un meilleur résultat.